# Dealing with special file formats

## JSON files

JSON is the de-facto standard nowadays to transfer any kind of structured data. JSON has the big advantage of being strictly defined (utf-8 encoding only), so it can be parsed easily, on all platforms, across all languages. Jupyter notebooks `*.ipynb` are purely JSON.

In Python, it is very easy to open an parse JSON files.

In [ ]:
import json
# read file
with open('01_interaction_with_the_file_system.ipynb', 'r', encoding='utf-8') as notebook:
    content = notebook.read()

# parse file
document = json.loads(content)

In [ ]:
document.keys()

In [ ]:
document['metadata']

## CSV files

In data science, CSV files are the gold standard, as they can be written and read by Excel ;-)

Of course, when dealing with _very_ large datasets, CSV files scale better than most other formats, they can be heavily compressed and quite easy to parse. The difficulty is to determine the right delimiter and the correct encoding, it is a trial and error process. You have to **read in the whole file** before you can be sure you've got the correct encoding.

In [ ]:
import csv

filepath = "data/data_laboratory_result.csv"

def read_csv(path, encoding, delimiter):
    with open(path, newline='', encoding=encoding) as csvfile:
        # this fails with UnicodeDecodeError if we have other decoding than utf-8:
        reader = csv.DictReader(csvfile, delimiter=delimiter)
        return [row for row in reader]

In [ ]:
leave_outer_loop = False
for encoding in ['utf-8', 'latin-1']:
    try:
        print(f"    Trying Encoding:   {encoding}")
        for delimiter in [',', ';', '\t']:
            print(f"    Trying delimiter:  {delimiter}")
            rows = read_csv(filepath, encoding, delimiter)
            if len(rows[0]) == 1: # we probably chose a wrong delimiter 
                continue
            else:
                delimiter_representation = "TAB" if delimiter == "\t" else delimiter
                print(f"      ✅ delimiter='{delimiter_representation}', encoding={encoding}")
                print(f"{len(rows)} lines read")
                leave_outer_loop = True
                break
        if leave_outer_loop:
            break

    except UnicodeDecodeError:
        print(f"    ❌ encoding failed for {encoding}")
        next

## CSV and Excel files, using `pandas`

In data science and when dealing with large tabular data, `Pandas` is the most popular tool to use. If you are familiar with the R language, you will find yourself at home! Pandas comes with a `read_csv()` method, but again, you need to know in advance what exact data format you are dealing with (i.e. delimiter and encoding).

The result is a so called **DataFrame**, which works like Excel on steroids and integrates very well with other data science libraries, like 

In [ ]:
import pandas as pd
filepath = "data/data_laboratory_result.csv"
df = pd.read_csv(filepath, delimiter=';', encoding='latin-1')

In [ ]:
df.head(10)

Of course, `pandas` can also read good, old Excel files, if needed. However, you need to install the `openpyxl` library to import them:

In [ ]:
!pip install openpyxl

In [ ]:
df2 = pd.read_excel('data/data_laboratory_result.xlsx')
df2.head(10)

## XML files: `BeautifulSoup` and `lxml`

XML files are becoming out of fashion these days, but we will have to deal with them anyway. The internal structure of any XML file is tree-like, and we need a practical way to move around the tree to extract the information we want. The most useful module to do that is **BeautifulSoup4**. It is not part of the standard library, so we have to intall it from pypi, using the `pip` utility:

In [ ]:
!pip install beautifulsoup4

As BeautifulSoup is just for navigating the XML file, we need also a parser to actually transform the file into an internal data structure. The standard parser is `lxml` which we can install using `pip`:

In [ ]:
!pip install lxml

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

filename = "data/20_Ms_215_1.xml"

parser = etree.XMLParser(dtd_validation=True, recover=True)          # set up the parser
tree = etree.parse(filename, parser)                                 # parse the file
unicode_string = etree.tostring(tree.getroot(), encoding='unicode')  # decode the dtd characters into unicode
soup = BeautifulSoup(unicode_string, 'lxml-xml')                     # feed BeautifulSoup with a unicode string, use lxml-xml parser

Once we have our soup ready, it is super-intuitive to navigate through the tree:

In [ ]:
title = soup.find('title')
title

In [ ]:
title.text

In [ ]:
title.parent

In [ ]:
title.parent.find('principal').text

In [ ]:
for child in title.children:
    print(child)

## YAML files

YAML files are becoming more popular these days, as Kubernetes' helm charts are written in YAML. And Ansible playbooks are written in YAML. It is even easier to write than JSON. Not to mention XML...

In Python, YAML files are as easy to read (and write) as JSON. `pyyaml` is the most popular module these days, it is unfortunately not part of the standard library yet and needs to be installed via `pip` first:

In [ ]:
!pip install pyyaml

In [ ]:
import yaml

with open('data/categories.yaml', 'r') as file_in:
    documents = yaml.full_load(file_in)    # use yaml.safe_load(file) for untrusted yaml files

    for item, doc in documents.items():
        print(item, ":", doc)

Writing YAML files is similarly easy, just create a datastructure and use the `yaml.dump()` method:

In [ ]:
categories = {
    "sports" : ['soccer', 'football', 'basketball', 'cricket', 'hockey', 'table tennis'],
    "countries" : ['Pakistan', 'USA', 'India', 'China', 'Germany', 'France', 'Spain']
}

In [ ]:
print(yaml.dump(categories))

## `toml` files

`toml` files are of particular interest in the python ecosystem, as `pyproject.toml` that describes a project or a package will soon become *the standard*. The main advantages over `yaml` or `json` files for configuration files are:

- very human readable
- no indentation necessary
- comments are allowed everywhere
- easily convertible into `json` or `yaml`
- will become part of the Python standard library in [python 3.11](https://docs.python.org/3.11/whatsnew/3.11.html), no installation necessary

Unless you are already using Python 3.11, you need to install the library:

In [ ]:
!pip install tomli

In [ ]:
import tomli
with open('data/configuration.toml', mode="rb") as file_in:
    configuration = tomli.load(file_in)   
    for item, doc in configuration.items():
        print(item, ":", doc)

## Dataframes with polars

[Polars](https://www.pola.rs) is a DataFrame library written in Rust (hence the `.rs` URL) with Python bindings. It is in many ways *much faster* than `pandas` and offers all kinds of manipulations for table-like data. Besides `.csv` files, it can read Excel, Parquet and JSON files as well as [read from external databases](https://pola-rs.github.io/polars-book/user-guide/io/database/).


Installation is – as always – easy:

In [ ]:
!pip install polars

In [ ]:
import polars as pl

query = (
    pl.scan_csv("data/iris.csv")         # lazy API, catches errors before processing and recommended for large files
    .filter(pl.col("sepal_length") > 5)
    .groupby("species")
    .agg(pl.all().sum())
)

df = query.collect()                     # query contains a LazyFrame, .collect() returns a proper DataFrame       
df

`query` shows the plan how polars is going to process the data:

In [ ]:
query

Or more formally:

In [ ]:
query.explain()